In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')

from datetime import timedelta

In [2]:
from config_paper import paper_key_id, paper_secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(paper_key_id, paper_secret_key,base_url)

In [4]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-05-13'

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        #dataframe['srsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.cash

ACTIVE


'100000'

In [7]:
symbollist=pd.read_csv('nasdaq-100-index-05-06-2020.csv')['Symbol']

In [8]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [9]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
#assetsToTrade = symbollist
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)
trade=True

In [10]:
trade=True
while trade==True:
    #
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
            
            
    #buy and sell        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df
                #api.get_barset(symbol,barTimeframe,limit=150).df
            data=populateindicators(returned_data)
        except:
            print(symbol)
        
        
        
#buy conditions -------
        #try:
        #set last time to buy stocks and when to start buying
        if currenttime <=1230 and currenttime >=630:    


        # Calculated trading indicators
        #populate exponential moving averages




         # Create the buy signal here
            if data['ultosc'][-3] < 31 and \
            data['ultosc'][-2] < 36 and \
            data['ultosc'][-2] > 31:
                buyord=True
                assetlist=api.list_positions()
                for asset in assetlist:
                    # if asset is in the list of positions we have already bought, then do not buy again
                    if asset.symbol == symbol:
                        buyord=False
                        print(asset.symbol,asset.qty)







                #if asset is not already bought, then buy             
                if buyord==True:
                    #cashBalance = account.daytrading_buying_power
                    #equity
                    cashBalance = float(account.buying_power)

                    targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))



                    targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))
                    buyreturned = api.submit_order(symbol,targetPositionSize,"buy","market","day")
                    print('Buy ',buyreturned)
            #except:
                #print('error with ',symbol)

    
    #get current time and shut off trading after 12
     
    if currenthour >= 13: 
        trade=False
        print('Good work, Turning off')
    

    
    

AGN
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '201b268b-cde8-4010-9fe3-41d2ac7f1e9c',
    'canceled_at': None,
    'client_order_id': '6e4675b9-73da-4e10-9350-00a23ef83354',
    'created_at': '2020-05-13T16:09:46.763151Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '5dfe6da7-1f27-41f3-bdc4-f04055fbfcb0',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '16',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-13T16:09:46.733901Z',
    'symbol': 'AMGN',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-05-13T16:09:46.763151Z'})
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '661531e0-6ce1-4238-b3d7-7247ddf8a99c',
    'canceled_at': None,
    'clie

Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'abedb6e6-42eb-4a23-aa05-05ce6500b503',
    'canceled_at': None,
    'client_order_id': '5c572d22-f9ad-442a-8aa6-3250f2d15d5e',
    'created_at': '2020-05-13T16:11:11.871596Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'f7866f81-33fc-4bd9-9d8c-c47e37090657',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '212',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-13T16:11:11.848153Z',
    'symbol': 'HOG',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-05-13T16:11:11.871596Z'})
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '945cfc7c-88e3-4db8-9f17-a4889b038cc5',
    'canceled_at': None,
    'client_o

Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'b9f7677c-965f-4074-b87b-cea5c848b6ce',
    'canceled_at': None,
    'client_order_id': '5e328882-1e3b-4e47-8680-7460e6fcf9a3',
    'created_at': '2020-05-13T16:11:23.7971Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '5c8510c3-fff8-4431-89c4-bd72d3a7cf1b',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '108',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-13T16:11:23.775514Z',
    'symbol': 'LYV',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-05-13T16:11:23.7971Z'})
MKTK
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '51fa61cb-6769-4e6e-b721-69348ce359f5',
    'canceled_at': None,
    'client_

Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '799b9c5a-54c1-4241-ba42-0dcba7d161bb',
    'canceled_at': None,
    'client_order_id': '6e55d0e5-bd87-495e-bd06-c3af6d3d14ac',
    'created_at': '2020-05-13T16:11:40.560751Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'fc511f0e-040a-475d-ae86-e668a536c39b',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '251',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-13T16:11:40.527197Z',
    'symbol': 'SLB',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-05-13T16:11:40.560751Z'})
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'b2b91a9b-3ecd-45ed-bf53-1f7d0613e1e8',
    'canceled_at': None,
    'client_o

Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'f1ca0594-6570-4d09-af05-abda3a9958ed',
    'canceled_at': None,
    'client_order_id': 'd299d47b-b233-4bf1-b1cf-9dc5473062a8',
    'created_at': '2020-05-13T16:12:00.143237Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '9517014e-a65f-4088-b3a1-a3d87d601b1d',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '379',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-13T16:12:00.087357Z',
    'symbol': 'APA',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-05-13T16:12:00.143237Z'})
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'ce4833e1-c561-492a-aace-7eb451d09e02',
    'canceled_at': None,
    'client_o

Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'e47acf22-5340-477a-a3ac-020b1ed34d12',
    'canceled_at': None,
    'client_order_id': '81611301-a954-46a8-9b02-de60fb0c8061',
    'created_at': '2020-05-13T16:12:32.774384Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'fba3ce0d-641a-47b5-976c-480c2db374cd',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '271',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-13T16:12:32.660144Z',
    'symbol': 'KMI',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-05-13T16:12:32.774384Z'})
MKTK
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'edc751a9-e628-40bd-9498-23ad7918269e',
    'canceled_at': None,
    'cli

sleep 3 seconds and retrying https://paper-api.alpaca.markets/v2/positions 3 more time(s)...


Buy  Order({   'asset_class': 'us_equity',
    'asset_id': '98dbc012-8ca2-4e68-b2f7-464fdc9bbf3c',
    'canceled_at': None,
    'client_order_id': '95e51df2-2f91-4f07-be96-7f0f387eb280',
    'created_at': '2020-05-13T16:12:58.067101Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': 'c5a16746-5855-47c0-a746-480d100abc17',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '195',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-13T16:12:58.036163Z',
    'symbol': 'UAL',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-05-13T16:12:58.067101Z'})
UTX

AGN
BRKB
BFB
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'c0a31d83-8c24-4408-966c-7710b5cc3de6',
    'canceled_at': N

APIError: cannot open a long buy while a short sell order is open

In [ ]:
data['emapulse'][-3]